In [ ]:
!pip install openai
import pandas as pd
import numpy as np
import itertools
import time
import openai
from tqdm.notebook import tqdm
import os, sys
openai.api_key = '##################'

In [35]:
df_sally = pd.read_csv('Results/Sally-Ann_result.csv', index_col = 0)
df_crayon = pd.read_csv('Results/Crayon-Box_result.csv', index_col = 0)

##Prompt for AutoGrader

The autograder is used to grade open ended generations, i.e. the qa and completion.

In [5]:
assess_smarties = """Reasoning:
          This is a Smarties story. Person A is Neila and Person B is Juanita. The container is a bag. Item X is plate and item Y is vest.
          Key facts:
          1. reality: A vest is in the bag because Neila found a vest in the bag.
          2. anti-reality: There supposed to be a plate in the bag because the label says plate.
          3. 1st order belief of Neila: Neila expects to find a vest in the bag because she already opened the bag and found a vest.
          4. 1st order belief of Juanita: Juanita expects fo find a plate in the bag because the label says plate and doesn't know there's actually a vest in the bag.
          5. 2nd order belief of Neila: Neila thinks that Juanita expects to find a plate in the bag, because she knows that Juanita doesn't know there's actually a vest in the bag.
          6. 2nd order belief of Juanita: Juanita thinks that Neila expects to find a plate in the bag, because she doesn't know that Neila already opened the bag and found a vest.
          The response to the prompts suggests that Juanita thinks Neila expects to find a plate, but instead she found a vest.
          The answer is correct because it is align with key facts 6.

          Grade:
          Correct."""

In [6]:
assess_sallyannee = """Reasoning:
          This is a Sally-Ann story. Person A is Neila and Person B is Juanita. The item is a towel. Location X is the closet and Location Y is the cabinet.
          Key facts:
          1. reality: The towel is currently in the cabinet because Neila moved it there.
          2. anti-reality: The towel was in the closet before.
          3. 1st order belief of Neila: Neila would look for the towel in the cabinet because she moved it there.
          4. 1st order belief of Juanita: Juanita would look for the towel in the closet because she didn't know that Neila moved it.
          5. 2nd order belief of Neila: Neila would think that Juanita looks for the towel in the closet because she knew that Juanita didn't see her move the towel to the cabinet.
          6. 2nd order belief of Juanita: Juanita would think that Neila looks for the towel in the closet because she didn't know that Neila already moved it.
          The response to the prompts suggests that Neila would think that Juanita would look for the towel in the cabinet.
          The answer is incorrect because it is contradictint key fact 5.

          Grade:
          Incorrect."""

In [7]:
system = """You are a helpful grader. You will be given a series of Prompt and Response pairs about Theory of Mind.
            You first need to reason about the context in the Prompt and figure out the key facts.
            There are two types of story in the context.
            The Sally-Ann story is about two people (Person A and Person B) found an item in location X. Person B left the room and Person A moved the item to location Y.
            Key facts for Sally-Ann story include:
            1. reality: the current location of the item, which is location Y.
            2. anti-reality: the previous location of the item, which is location X.
            3. 1st order belief of Person A: what Person A thinks the current location of the item, which is location Y.
            4. 1st order belief of Person B: what Person B thinks the current location of the item, whic is location X.
            5. 2nd order belief of Person A: what Person A thinks that Person B thinks the current location of the item, which is location X.
            6. 2nd order belief of Person B: what Person B thinks that Person A thinks the current location of the item, which is location Y.
            The Smarties story is about Person A found a container. The label on the container says item X, but there's no item X in the container. Person A opened the container and found item Y. Later, Person B entered the room and saw the container.
            Key facts for Smarties story include:
            1. reality: what's inside the container, which is item Y.
            2. anti-reality: what supposed to be in the container according to the label, which is item X.
            3. 1st order belief of Person A: what Person A expects to find in the container, which is item Y (because Person A already opened the bag and found Y).
            4. 1st order belief of Person B: what Person B expects to find in the container, which is item X.
            5. 2nd order belief of Person A: what Person A thinks that Person B expects to find in the container, which is item X.
            6. 2nd order belief of Person B: what Person B thinks that Person A thinks is in the container, which is item X.

            After identifying the key facts of the context, you can take a look at the response to see if that is align with the key facts.
            If the response align with the key facts, it can be graded as correct. Otherwise it's incorrect. """

In [8]:
user_sallyanne = 'Prompt:\n' + df_sally['qa_prompt'][4] + '\nResponse:\n' + df_sally['qa_turbo_pred'][4]
user_smarties = 'Prompt:\n' + df_crayon['comp_prompt'][4] + '\nResponse:\n' + df_crayon['comp_turbo_pred'][4]

In [9]:
input = [{"role": "system", "content": system},
         {"role": "user", "content": user_sallyanne},
         {"role": "assistant", "content": assess_sallyannee },
         {"role": "user", "content": user_smarties},
         {"role": "assistant", "content": assess_smarties}]

##Example Use:

Grading the qa_answer in sally-anne. The turbo model's response is wrong.

In [15]:
print("original answer:")
print('Prompt:\n' + df_sally['qa_prompt'][8] + '\nShort Answer:\n' + df_sally['short_answer'][8] + '\nResponse:\n' + df_sally['qa_turbo_pred'][8])

original answer:
Prompt:
Answer the question based on context:

Context:Sabra and Hillary were hanging out in the bedroom. They saw a briefcase and a bucket. They found a mitten in the briefcase. Hillary left the bedroom. Sabra moved the mitten to the bucket.
Question:
After Hillary came back to the bedroom, where would Hillary look for the mitten?
Short Answer:
briefcase.
Response:
Hillary would most likely look in the bucket where Sabra moved the mitten.


In [10]:
input.append({"role": "user", "content": 'Prompt:\n' + df_sally['qa_prompt'][8] + '\nResponse:\n' + df_sally['qa_turbo_pred'][8]})

In [11]:
grade = openai.ChatCompletion.create(
      messages=input,
      model = 'gpt-4-0613',
      temperature=0,
      max_tokens= 500,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0)

In [13]:
##Human Grade to this answer. 0 means wrong.
df_sally['qa_turbo_grad'][8]

0

In [12]:
##Autograder's result is align with human grader.
print(grade['choices'][0]['message']['content'])

Reasoning:
          This is a Sally-Ann story. Person A is Sabra and Person B is Hillary. The item is a mitten. Location X is the briefcase and Location Y is the bucket.
          Key facts:
          1. reality: The mitten is currently in the bucket because Sabra moved it there.
          2. anti-reality: The mitten was in the briefcase before.
          3. 1st order belief of Sabra: Sabra would look for the mitten in the bucket because she moved it there.
          4. 1st order belief of Hillary: Hillary would look for the mitten in the briefcase because she didn't know that Sabra moved it.
          5. 2nd order belief of Sabra: Sabra would think that Hillary looks for the mitten in the briefcase because she knew that Hillary didn't see her move the mitten to the bucket.
          6. 2nd order belief of Hillary: Hillary would think that Sabra looks for the mitten in the briefcase because she didn't know that Sabra already moved it.
          The response to the prompts suggests tha

## Generate auto grading results for qa and comp

In [16]:
def get_grade(input):
  raw_grade = openai.ChatCompletion.create(
    messages=input,
    model = 'gpt-4-0613',
    temperature=0,
    max_tokens= 500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0)
  return raw_grade['choices'][0]['message']['content']

In [33]:
input = [{"role": "system", "content": system},
         {"role": "user", "content": user_sallyanne},
         {"role": "assistant", "content": assess_sallyannee},
         {"role": "user", "content": user_smarties},
         {"role": "assistant", "content": assess_smarties}]
def auto_grader(df, task, model):
  col_prompt = '_'.join([task, 'prompt'])
  col_pred = '_'.join([task, model, 'pred'])
  col_autograde_num = '_'.join([task, model, 'autograde'])
  col_autograde_full= '_'.join([task, model, 'autograde', 'full'])
  df[col_autograde_num] = ''
  df[col_autograde_full] = ''
  max_attempts = 10
  for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if pd.isna(row[col_autograde_num]) or row[col_autograde_full] == '':
      for attempt in range(max_attempts):
        prompt = row[col_prompt]
        pred = row[col_pred]
        user = {"role": "user", "content": "Prompt:\n" + prompt + "\nResponse:\n" + pred}
        input.append(user)
        try:
          full_grade = get_grade(input)
          df.at[index, col_autograde_full] = full_grade
          score = full_grade.split('\n')[-1].lstrip().rstrip()
          print('here5')
          if score.lower() == "incorrect":
            df.at[index, col_autograde_num] = 0
          else:
            df.at[index, col_autograde_num] = 1
        except Exception as e:
          print(f"Encountered an error at index {index}. Retrying attempt {attempt+1}...")
          time.sleep(5)
      else:
        print(f"Failed to get a response after {max_attempts} attempts at index {index}. Moving on to next row.")
  return df

In [ ]:
for task in ['qa', 'comp']:
  for model in ['davinci', 'turbo', 'gpt4']:
    df_sally = auto_grader(df_sally, task, model)
    df_crayon = auto_grader(df_crayon, task, model)